In [ ]:
!pip install oauthlib
!pip install requests_oauthlib
!pip install sentinelhub

In [ ]:
import pandas as pd
from PIL import Image
from datetime import datetime, timedelta
import requests
from data_mining.get_sat2_images import call_sentinel
# from data_mining.read_firenews import fetch_fire_news
import data_mining.read_weather as weather
from datetime import datetime, timedelta
import tensorflow as tf
import pandas as pd
from keras.models import Model
from keras.layers import LSTM, Dense, Input, TimeDistributed, Flatten, Concatenate, Reshape, Masking
import os
from data_to_tensor import location2sentence
from tensorflow.keras.utils import Sequence
import numpy as np


# Gather data
## dates

### fetch from fire news

In [ ]:
# start_date="2022-05-01"
# end_date="2025-06-10"
# fire_data=fetch_fire_news(start_date,end_date,verbose=True)
# fetch_fire_news(start_date,end_date,save=True)

## 